<a href="https://colab.research.google.com/github/franklinwillemen/Sherlock-Hemlock-Graph-Based-NER/blob/main/data_build_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Create a symbolic link to the Colab Notebooks directory
nb_path = '/content/notebooks'

# Check if the symbolic link or directory already exists
if not os.path.exists(nb_path):
    os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)

# Add the symbolic link to the system path
sys.path.insert(0, nb_path)

Mounted at /content/drive


In [9]:
import logging
import transformers as t
import datasets
from datasets import load_dataset
import torch
import numpy as np
import pandas as pd
import spacy

In [10]:
dataset = load_dataset("joelito/mapa")

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 180.11it/s]


Dataset json downloaded and prepared to C:/Users/Frank/.cache/huggingface/datasets/joelito___json/joelito--mapa-b7771d6b2e7c8e8e/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 92.57it/s]


In [11]:
df = pd.DataFrame(dataset["train"].to_pandas())

In [19]:
# List of language codes to drop
languages_to_drop = ["ga", "mt", "sk"]

# Drop the rows with the specified language codes
mapa_df = df[~df['language'].isin(languages_to_drop)]

mapa_df = mapa_df.drop(["type", "file_name", "sentence_number"], axis=1)

# Reset the index of the filtered DataFrame
mapa_df.reset_index(drop=True, inplace=True)

In [20]:
nl = mapa_df[mapa_df["language"] == "nl"]

In [13]:
!python -m spacy download nl_core_news_lg

     ---------------------------------------- 0.0/568.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/568.1 MB ? eta -:--:--
     -------------------------------------- 0.0/568.1 MB 259.2 kB/s eta 0:36:32
     -------------------------------------- 0.0/568.1 MB 259.2 kB/s eta 0:36:32
     -------------------------------------- 0.0/568.1 MB 259.2 kB/s eta 0:36:32
     -------------------------------------- 0.0/568.1 MB 259.2 kB/s eta 0:36:32
     -------------------------------------- 0.0/568.1 MB 259.2 kB/s eta 0:36:32
     -------------------------------------- 0.0/568.1 MB 259.2 kB/s eta 0:36:32
     -------------------------------------- 0.0/568.1 MB 259.2 kB/s eta 0:36:32
     -------------------------------------- 0.1/568.1 MB 209.5 kB/s eta 0:45:11
     -------------------------------------- 0.1/568.1 MB 293.2 kB/s eta 0:32:17
     -------------------------------------- 0.2/568.1 MB 389.1 kB/s eta 0:24:20
     -------------------------------------- 0.3/568.1 

In [15]:
nlp = spacy.load("nl_core_news_lg")

In [51]:
def extract_pos_tags_deps(sentence):
    try:
        doc = nlp(" ".join(sentence))
        pos = [token.pos_ for token in doc]
        tag = [token.tag_ for token in doc]
        dep = [token.dep_ for token in doc]
    except:
        pos, tag, dep = [], [], []
    return pos, tag, dep

In [52]:
nl[['pos', 'tag', 'dep']] = nl['tokens'].apply(lambda x: pd.Series(extract_pos_tags_deps(x)))

C:\Users\Frank\AppData\Local\Temp\ipykernel_5876\1918081245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nl[['pos', 'tag', 'dep']] = nl['tokens'].apply(lambda x: pd.Series(extract_pos_tags_deps(x)))


In [54]:
nl["tag"]

18698    [N|soort|ev|basis|onz|stan, VZ|init, LID|onbep...
18699    [LET, SPEC|deeleigen, SPEC|deeleigen, LET, TW|...
18700    [N|eigen|ev|basis|onz|stan, LET, N|eigen|ev|ba...
18701     [WW|od|prenom|met-e, N|soort|ev|basis|zijd|stan]
18702         [N|soort|ev|basis|zijd|stan, SPEC|deeleigen]
                               ...                        
20084    [LET, BW, LET, LID|onbep|stan|agr, N|soort|ev|...
20085    [LET, BW, LET, LID|onbep|stan|agr, ADJ|prenom|...
20086    [TW|hoofd|vrij, LET, LID|onbep|stan|agr, ADJ|p...
20087                                   [N|soort|mv|basis]
20088    [LET, N|eigen|ev|basis|onz|stan, LET, N|eigen|...
Name: tag, Length: 1391, dtype: object

In [50]:
" ".join(nl["tokens"][18698])


'Verzoek om een prejudiciële beslissing , ingediend door de Rechtbank Amsterdam ( Nederland ) op 8 mei 2018 – Openbaar Ministerie tegen SF'